# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921691000913                   -0.69    5.8         
  2   -7.926165916580       -2.35       -1.22    1.0    228ms
  3   -7.926835844143       -3.17       -2.37    1.6    270ms
  4   -7.926861478311       -4.59       -2.99    2.8    264ms
  5   -7.926861621870       -6.84       -3.30    2.1    227ms
  6   -7.926861664710       -7.37       -3.69    1.4    242ms
  7   -7.926861680594       -7.80       -4.41    1.2    201ms
  8   -7.926861681816       -8.91       -4.96    2.4    233ms
  9   -7.926861681852      -10.45       -5.13    2.0    225ms
 10   -7.926861681872      -10.71       -5.97    1.1    217ms
 11   -7.926861681873      -11.97       -6.93    2.2    237ms
 12   -7.926861681873      -13.62       -7.25    3.1    259ms
 13   -7.926861681873   +  -14.75       -7.77    1.4    220ms
 14   -7.926861681873   +  -15.05       -8.67    2.1    226ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921729185034                   -0.69    5.6         
  2   -7.926168680176       -2.35       -1.22    1.0    213ms
  3   -7.926836952866       -3.18       -2.37    1.6    229ms
  4   -7.926861526280       -4.61       -3.01    2.9    308ms
  5   -7.926861635190       -6.96       -3.34    2.0    215ms
  6   -7.926861667660       -7.49       -3.74    1.6    210ms
  7   -7.926861680665       -7.89       -4.41    1.0    196ms
  8   -7.926861681823       -8.94       -5.01    2.4    265ms
  9   -7.926861681858      -10.45       -5.20    1.9    224ms
 10   -7.926861681872      -10.87       -5.95    1.1    199ms
 11   -7.926861681873      -12.01       -6.94    2.4    237ms
 12   -7.926861681873      -13.59       -7.37    2.8    261ms
 13   -7.926861681873   +    -Inf       -8.35    1.8    213ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921627920561                   -0.69    5.8         
  2   -7.926167794491       -2.34       -1.22    1.0    253ms
  3   -7.926839932235       -3.17       -2.37    1.9    239ms
  4   -7.926864893217       -4.60       -3.00    2.5    251ms
  5   -7.926865032067       -6.86       -3.30    1.9    215ms
  6   -7.926865073232       -7.39       -3.65    1.6    211ms
  7   -7.926865091496       -7.74       -4.36    1.1    218ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       
